In [62]:
import paramiko
from getpass import getpass

# Create the prompt for the password
password = getpass(prompt="Please Enter your SSH password: ")

# Hist Information
host = "192.168.6.71"
port = 2222
username = "dylan.navarro@cyber.local"


try: 
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    # Open the SSH session
    ssh.connect(host, port, username, password)
except paramiko.AuthenticationException:
    print("Authentication Failed.")

In [54]:
# Start SFTP Session
from threading import local

# Start the SFTP Session
sftp = ssh.open_sftp()

# Make the file executable
command_chmod = "chmod +x ./kraken"
ssh.exec_command(command_chmod)

# Command to run the file
command_kraken = "sudo -S /home/ubuntu/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin > navarro_kraken.txt"
# Run the Command 
stdin, stdout, stderr = ssh.exec_command(command_kraken)
# Pass user password to the sudo prompt
stdin.write(password + "\n")
stdin.flush()
# Hits the enter for the end of the command to finish
stdin.write("\n")
# The command above seems to be having issues when executing. I think it's getting terminated before completion
# This should make it wait till it finishes.
exit_status = stdout.channel.recv_exit_status()

# Pull the remote output file.
# Remote File
remote_file = "navarro_kraken.txt"
# Lets get the results
sftp.get(remote_file, "navarro_kraken.txt")

In [55]:
# Get matching processes out of the output file.
import re
pid_list = []
# Need to remove the cringe color code stuff.
no_ansi = []
# Regex to remove the ansi color settings
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
# Open the results file
with open("navarro_kraken.txt") as kraken_results:
    # Read the file
    lines = kraken_results.readlines()
    # loop through each line in the file
    for line in lines:
        # Remove the ansi color stuff
        line = ansi_escape.sub('', line)
        # Add the cleaned line to the no_ansi list
        no_ansi.append(line)
# Loop through the cleaned list
for line in no_ansi:
    # Search the item in the list for a PID string
    search = re.search(".*pid.*=.*", line)
    # If PID found then append it to the pid_list array.
    if search:
        # Pulls the specific PID out of the string
        pid = re.findall(r'pid=[0-9]{1,10}',line)
        if (len(pid) > 0):
            cleaned = pid[0].split("=")
            pid_list.append(cleaned[1])
print(pid_list)

['519423', '561559']


In [56]:
# Now that we have a list of PIDs lets get some informaiton from them. 
pid_string = ""
for pid in pid_list:
    if pid == pid_list[-1]:
        pid_string += pid
    else:
        pid_string += pid
        pid_string += ","
command_losf = f"sudo -S lsof -p {pid_string} >> navarro_lsof.txt"
# Run the Command 
stdin, stdout, stderr = ssh.exec_command(command_losf)
# Pass user password to the sudo prompt
stdin.write(password + "\n")
stdin.flush()
exit_status = stdout.channel.recv_exit_status()

# Get the file paths
command_awk = "awk '{print $9}' navarro_lsof.txt > navarro_files.txt"
# Run the Command 
stdin, stdout, stderr = ssh.exec_command(command_awk)
exit_status = stdout.channel.recv_exit_status()

# Pull the remote output files.
# Remote File
remote_file_lsof = "navarro_lsof.txt"
# Lets get the results
sftp.get(remote_file_lsof, remote_file_lsof)
# Remote File
remote_file_files = "navarro_files.txt"
# Lets get the results
sftp.get(remote_file_files, remote_file_files)

    

In [61]:
# Time to get the files from the remote system
file_paths = []
with open("navarro_files.txt") as file_list:
    # Read the file
    lines = file_list.readlines()
    # loop through each line in the file
    for line in lines:
        # Check to see if it's a valid file path and not just /
        #if line[0] == "/" and len(line) > 2:
        # Lets target user directories since the TA makes users
        if line[0] == "/" and line.split("/")[1] == "home":
            # Check for duplicates
            if line[0:-1] not in file_paths:
                # Remove the new line character at the end.
                file_paths.append(line[0:-1])
print(file_paths)
for file in file_paths:
    file_name = file.split("/")[-1][0:-1]
    print(f"Moving file {file} to /home/dylan.navarro/send/{file_name}")
    command_cp = f"sudo -S cp {file} /home/dylan.navarro/send/{file_name}"
    stdin, stdout, stderr = ssh.exec_command(command_cp)
    stdin.write(password + "\n")
    stdin.flush()
    exit_status = stdout.channel.recv_exit_status()
    print("Done")
    print(f"Changing Owner /home/dylan.navarro/send/{file_name}")
    command_chown = f"sudo -S chown dylan.navarro /home/dylan.navarro/send/{file_name}"
    stdin, stdout, stderr = ssh.exec_command(command_chown)
    stdin.write(password + "\n")
    stdin.flush()
    exit_status = stdout.channel.recv_exit_status()
    print("Done")

    try:   
        remote_file = f"/home/dylan.navarro/send/{file_name}"
        local_file = "downloads\\" + file_name
        print(local_file)
        # Lets get the results
        sftp.get(remote_file, local_file)
    except FileNotFoundError:
        print(f"{file_name} not found. Maybe it's a dir?")


['/home/p.dalton', '/home/p.dalton/.d.log', '/home/p.dalton/d.pid']
Moving file /home/p.dalton to /home/dylan.navarro/send/p.dalto
Done
Changing Owner /home/dylan.navarro/send/p.dalto
Done
downloads\p.dalto
p.dalto not found. Maybe it's a dir?
Moving file /home/p.dalton/.d.log to /home/dylan.navarro/send/.d.lo
Done
Changing Owner /home/dylan.navarro/send/.d.lo
Done
downloads\.d.lo
Moving file /home/p.dalton/d.pid to /home/dylan.navarro/send/d.pi
Done
Changing Owner /home/dylan.navarro/send/d.pi
Done
downloads\d.pi


In [69]:
# Task 5
# Get Users
command_get_users = "sudo -S cat /etc/passwd | awk -F ':' '{print $1}' > navarro_users.txt"
stdin, stdout, stderr = ssh.exec_command(command_get_users)
stdin.write(password + "\n")
stdin.flush()
exit_status = stdout.channel.recv_exit_status()

# Downlod Users File
sftp.get("navarro_users.txt", "navarro_users.txt")

# Parse the users
users = []
with open("navarro_users.txt") as users_list:
    # Read the file
    lines = users_list.readlines()
    # loop through each line in the file
    for line in lines:
        # Return users that match the naming convention
        if re.search("[A-Za-z]{1}\.[A-Za-z]{5}", line):
            users.append(line)

for user in users:
    pass

# I don't think I actually need to script this. Just need to answer the questions.


<paramiko.ChannelFile from <paramiko.Channel 7 (open) window=2097139 -> <paramiko.Transport at 0xd9cc2940 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>


### 1. What is the process name?

$ ps aux

/lib/systemd/systemd --user

(sd-pam)

[go-daemon sample]



### 2. Which user is running the process?

p.dalton

### 3. Has the user logged in before via SSH?

Yes.

### 4. When did the user login?


$ lastlog -u p.dalton

```Tue Oct 25 23:23:31 -0400 2022```


### 5. How long was the login session?

$last p.dalton

9 Minutes


### 6. What is located in the unauthorized user’s home directory?

/home/p.dalton/:
.bash_history, .cache, .d.log, aqwerpuqwerj, d.pid, test-lk


/home/p.dalton/.cache:
motd.legal-displayed


/home/p.dalton/test-lk:
.d.log, d.pid


In [ ]:
# Close the SSH Session
ssh.close()

# Reflections

1. Based on what you discovered and the information in the threat report, what is your conclusion about the host you analyzed?
It seems that the host has been compromised by the threat actor. 

2. What did you like the most and least about this assignment?
I don't like how the directions were kinda confusing. I like how this is somewhat realistic. That being said I don't think I would program this in this style. I would prefer to use ansible which uses the paramiko. Having to write all the connection stuff manually is kinda annoying when there is already a tool out there that does it and I have experience with it. 

3. What additional questions do you have?
N/A